In [3]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
experiments = client.search_experiments()
for exp in experiments:
    print(exp)

<Experiment: artifact_location='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1733127854674, experiment_id='1', last_update_time=1733127854674, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1733127377347, experiment_id='0', last_update_time=1733127377347, lifecycle_stage='active', name='Default', tags={}>


In [6]:
client.create_experiment(name="my-cool-experiment")

'2'

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 58c8094a5c504e6ba0f9a024a982c28b, rmse: 5.4241
run id: 32ecfc37cc4842c195a3c2257bcd6c8a, rmse: 5.4636
run id: 61d9a191165b4daa82fb58552ffdb4f8, rmse: 5.4856
run id: 95624d20d4674b9181f13c11b4b6343e, rmse: 5.7761


In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
run_id = "58c8094a5c504e6ba0f9a024a982c28b"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1733130680918, current_stage='None', description=None, last_updated_timestamp=1733130680918, name='nyc-taxi-regressor', run_id='58c8094a5c504e6ba0f9a024a982c28b', run_link=None, source='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1/58c8094a5c504e6ba0f9a024a982c28b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [12]:
client = MlflowClient()
model_name = "nyc-taxi-regressor"

# Search for all registered models
registered_models = client.search_registered_models(f"name='{model_name}'")
for model in registered_models:
    for version in model.latest_versions:
        print(f"Version: {version.version}, Stage: {version.current_stage}")

Version: 2, Stage: None


In [ ]:
# Old version and we will get a warning if we use this way so please use below.
model_version = 2
new_stage = "Staging"

client.set_model_version_tag(
    name=model_name,
    version=model_version,
    key="lifecycle_stage",
    value=new_stage
)
print(f"Model version {model_version} tagged with lifecycle_stage: {new_stage}")

Model version 2 tagged with lifecycle_stage: Staging


In [15]:
# Search for all registered models
registered_models = client.search_registered_models(f"name='{model_name}'")
for model in registered_models:
    for version in model.latest_versions:
        print(f"Version: {version.version}, Stage: {version.current_stage}")

Version: 1, Stage: None
Version: 2, Stage: Staging


In [16]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1733130680918, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-12-02', last_updated_timestamp=1733131377672, name='nyc-taxi-regressor', run_id='58c8094a5c504e6ba0f9a024a982c28b', run_link=None, source='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1/58c8094a5c504e6ba0f9a024a982c28b/artifacts/model', status='READY', status_message=None, tags={'lifecycle_stage': 'Staging'}, user_id=None, version=2>